In [10]:
!pip install flask
!pip install flask-ngrok

In [13]:
import sqlite3
import pandas as pd
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
from threading import Thread
from concurrent.futures import ThreadPoolExecutor

In [14]:
db_path = '/content/drive/MyDrive/Colab Notebooks/COMPUTAÇÃO NEUROMÓRFICA/GS - 2/health.db'
file_path = '/content/drive/MyDrive/Colab Notebooks/COMPUTAÇÃO NEUROMÓRFICA/GS - 2/Sleep_health_and_lifestyle_dataset.csv'

In [15]:
conn = sqlite3.connect(db_path)

cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS sleepHealth (
        id INTEGER PRIMARY KEY,
        gender VARCHAR,
        age INTEGER,
        occupation VARCHAR,
        sleepDuration REAL,
        qualitySleep INTEGER,
        physicalActivityLevel INTEGER,
        stressLevel INTEGER,
        BMICategory VARCHAR,
        bloodPressure VARCHAR,
        heartRate INTEGER,
        dailySteps INTEGER,
        sleepDisorder VARCHAR
    )
''')

In [16]:
def inserir_chunk(chunk):
    conexao = sqlite3.connect(db_path)
    cursor = conexao.cursor()

    for indice, linha in chunk.iterrows():
        valores = tuple(linha[1:])
        cursor.execute('INSERT INTO sleepHealth (gender, age, occupation, sleepDuration, qualitySleep, physicalActivityLevel, stressLevel, BMICategory, bloodPressure, heartRate, dailySteps, sleepDisorder) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', valores)

    conexao.commit()
    conexao.close()

dados = pd.read_csv(file_path, sep=',', encoding='latin-1')

chunk_size = 1000
chunks = [dados[i:i + chunk_size] for i in range(0, len(dados), chunk_size)]

# Inserir chunks em paralelo
with ThreadPoolExecutor() as executor:
    executor.map(inserir_chunk, chunks)

In [ ]:
app = Flask(__name__)

run_with_ngrok(app)

def getsleepHealth():
  conn = sqlite3.connect(db_path)
  cursor = conn.cursor()
  cursor.execute("SELECT * FROM sleepHealth")
  resultados = cursor.fetchall()
  conn.close()

  dados = []
  for linha in resultados:
      dados.append({"id": linha[0], "gender": linha[1], "age": linha[2], "occupation": linha[3], "sleepDuration": linha[4], "qualitySleep": linha[5], "physicalActivityLevel": linha[6], "stressLevel": linha[7], "BMICategory": linha[8], "bloodPressure": linha[9], "heartRate": linha[10], "dailySteps": linha[11], "sleepDisorder": linha[12]})

  return jsonify(dados), 200

@app.route('/')
def hello_world():
    return jsonify({"message": "GS - Computacao neuromorfica"})

@app.route('/api/sleepHealth', methods=['GET'])
def getData():
    try:
      thread = Thread(target=getsleepHealth)
      thread.start()

      thread.join()

      return getsleepHealth()

    except Exception as e:
        return jsonify({"erro": str(e)}), 400

@app.route('/api/sleepHealth', methods=['POST'])
def createData():
    try:
        gender = request.json['gender']
        age = request.json['age']
        occupation = request.json['occupation']
        sleepDuration = request.json['sleepDuration']
        qualitySleep = request.json['qualitySleep']
        physicalActivityLevel = request.json['physicalActivityLevel']
        stressLevel = request.json['stressLevel']
        BMICategory = request.json['BMICategory']
        bloodPressure = request.json['bloodPressure']
        heartRate = request.json['heartRate']
        dailySteps = request.json['dailySteps']
        sleepDisorder = request.json['sleepDisorder']

        conn = sqlite3.connect(db_path)
        conn.execute('INSERT INTO sleepHealth (gender, age, occupation, sleepDuration, qualitySleep, physicalActivityLevel, stressLevel, BMICategory, bloodPressure, heartRate, dailySteps, sleepDisorder) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', (gender, age, occupation, sleepDuration, qualitySleep, physicalActivityLevel, stressLevel, BMICategory, bloodPressure, heartRate, dailySteps, sleepDisorder))
        conn.commit()
        conn.close()

        return jsonify({"mensagem": "Dado criado com sucesso!"}), 200

    except Exception as e:
        return jsonify({"erro": str(e)}), 400

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f2e6-34-125-164-93.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
